# Download ans Save Data: Continuous
### Xiaotao Yang
This notebook shows examples of calling seisgo.utils.getdata() to download seismic data and save them to files.

## Step 0. Load needed packages.
Some functions are imported from the utils.py.

In [1]:
#import needed packages.
from seisgo import utils
import sys
import time
import scipy
import obspy
import pyasdf
import datetime
import os, glob
import numpy as np
from obspy import UTCDateTime
from obspy.core import Stream,Trace
from IPython.display import clear_output
from obspy.clients.fdsn import Client

## Step 1. Set global parameters
These parameters will control the downloading procedure.

In [2]:
"""
1. Station parameters
"""
rawdatadir = '../data/raw_test'
if not os.path.isdir(rawdatadir): os.mkdir(rawdatadir)
    
cleantargetdir=True #change to False or remove/comment this block if needed.
if cleantargetdir:
    dfiles1 = glob.glob(os.path.join(rawdatadir,'*.h5'))
    if len(dfiles1)>0:
        print('Cleaning up raw data directory before downloading ...')
        for df1 in dfiles1:
            os.remove(df1)

source='IRIS'
client=Client(source)
# get data from IRIS web service
inet="TA"
stalist=["G05D","I04D"]#["G03A","J35A","J44A","J65A"]
chanlist=['HHZ','BHZ']

starttime = "2012_02_02_0_0_0"
endtime   = "2012_02_05_0_0_0"
inc_hours = 8

"""
2. Preprocessing parameters
"""
rmresp=True #remove instrument response
# parameters for butterworth filter
samp_freq=10
pfreqmin=0.002
pfreqmax=samp_freq/2

# prefilter information used when removing instrument responses
f1 = 0.95*pfreqmin;f2=pfreqmin
if 1.05*pfreqmax > 0.48*samp_freq:
    f3 = 0.45*samp_freq
    f4 = 0.48*samp_freq
else:
    f3 = pfreqmax
    f4= 1.05*pfreqmax
pre_filt  = [f1,f2,f3,f4]


"""
3. Download by looping through datetime list.
***** The users usually don't need to chance the following lines ****
"""
dtlist = utils.split_datetimestr(starttime,endtime,inc_hours)
print(dtlist)
for idt in range(len(dtlist)-1):
    sdatetime = obspy.UTCDateTime(dtlist[idt])
    edatetime   = obspy.UTCDateTime(dtlist[idt+1])

    fname = os.path.join(rawdatadir,dtlist[idt]+'T'+dtlist[idt+1]+'.h5')

    """
    Start downloading.
    """
    
    for ista in stalist:
        print('Downloading '+inet+"."+ista+" ...")
        """
        3a. Request data.
        """
        for chan in chanlist:
            try:
                t0=time.time()
                tr,sta_inv = utils.getdata(inet,ista,sdatetime,edatetime,chan=chan,source=source,
                                           samp_freq=samp_freq,plot=False,rmresp=rmresp,rmresp_output='DISP',
                                           pre_filt=pre_filt,sacheader=True,getstainv=True)
                
                ta=time.time() - t0
                print('  downloaded '+inet+"."+ista+"."+chan+" in "+str(ta)+" seconds.")
                """
                3b. Save to ASDF file.
                """
                tags=[]
                if len(tr.stats.location) == 0:
                    tlocation='00'
                else:
                    tlocation=tr.stats.location

                tags.append(tr.stats.channel.lower()+'_'+tlocation.lower())

                print('  saving to '+fname)
                utils.save2asdf(fname,Stream(traces=[tr]),tags,sta_inv=sta_inv)  
            except Exception as e:
                print(str(e))
            
        

['2012_02_02_00_00_00', '2012_02_02_08_00_00', '2012_02_02_16_00_00', '2012_02_03_00_00_00', '2012_02_03_08_00_00', '2012_02_03_16_00_00', '2012_02_04_00_00_00', '2012_02_04_08_00_00', '2012_02_04_16_00_00', '2012_02_05_00_00_00']
station 7D.G03A --> seismic channel: BHZ
  downsamping from 50 to 10
  removing response using inv for 7D.G03A.BHZ
  downloaded 7D.G03A.BHZ in 4.829556941986084 seconds.
  saving to ../data/raw_test/2012_02_02_00_00_00T2012_02_02_08_00_00.h5
station 7D.G03A --> seismic channel: BHZ
  downsamping from 50 to 10
  removing response using inv for 7D.G03A.BHZ
  downloaded 7D.G03A.BHZ in 4.547628164291382 seconds.
  saving to ../data/raw_test/2012_02_02_08_00_00T2012_02_02_16_00_00.h5
station 7D.G03A --> seismic channel: BHZ
  downsamping from 50 to 10
  removing response using inv for 7D.G03A.BHZ
  downloaded 7D.G03A.BHZ in 4.405111074447632 seconds.
  saving to ../data/raw_test/2012_02_02_16_00_00T2012_02_03_00_00_00.h5
station 7D.G03A --> seismic channel: BHZ
  